In [1]:
import sys
import torch
import numpy as np
from tqdm import tqdm
sys.path.append('../../model/geometric_temporal/')
sys.path.append('../../dataset/')

In [2]:
from recurrent import LSTMGCNModel
from gdelt_dataset import GDELTDatasetLoader
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal
seed = torch.random.manual_seed(52)

In [3]:
dtype = torch.float
device = torch.device("cpu")

In [4]:
loader = GDELTDatasetLoader('../../data/dataset/gdelt_data_2.pkl', 1000)
dataset = loader.get_dataset()

In [5]:
model = LSTMGCNModel(7, 5, 1, 3)

optimizer = torch.optim.Adam(model.parameters(), lr=5)

model.train()

for epoch in tqdm(range(10)):
    cost = 0
    for time, snapshot in enumerate(dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        inner_cost = 0
        for i, j in zip(snapshot.edge_index[0], snapshot.edge_index[1]):
            pred = torch.dot(y_hat[i], y_hat[j])
            true_val = snapshot.y[i][j]
            exp = torch.pow(pred-true_val, 2)
            inner_cost += exp
        inner_cost /= (snapshot.edge_index.shape[0])
        if not torch.isnan(inner_cost):
            cost = cost + inner_cost
    cost = cost / (time+1)
    print(cost)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

tensor(86.6306, grad_fn=<DivBackward0>)


 10%|████████▎                                                                          | 1/10 [00:06<01:01,  6.82s/it]

0.0


AttributeError: 'float' object has no attribute 'backward'